# <font color = "red">Classificação de Risco de Crédito</font>

## Uma análise Comparativa Para Prever Inadimplência 

<font color = "red">Problema de Negócio:</font> A partir de dados históricos, irei verificar os padrões que levaram os clientes de uma instituição financeira, a adentrarem, ou não, na situação de inadimplência, para posteriormente criar um 


<font color = "red">Meta de Acurácia Final:</font> ACC mínimo de 88%

In [131]:
# Imports 
import sys
import os
import pandas as pd
import numpy as np


sys.path.append(os.path.abspath('..'))
from utils import *

In [132]:
df_original = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/risco_credito_original.csv")

In [133]:
df_original.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')

In [134]:
df_original.shape

(32581, 12)

In [135]:
df_original.sample(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
3832,23,32993,RENT,6.0,MEDICAL,B,4000,11.71,1,0.12,N,4
2267,26,31200,MORTGAGE,6.0,PERSONAL,A,9250,7.90,0,0.30,N,2
22790,31,58000,OWN,15.0,MEDICAL,C,14500,13.06,0,0.25,N,7
31623,44,115000,MORTGAGE,3.0,HOMEIMPROVEMENT,B,12000,NaN,0,0.10,N,14
10597,25,69000,MORTGAGE,3.0,VENTURE,A,6000,7.88,0,0.09,N,2
29568,45,22178,RENT,3.0,VENTURE,A,2500,5.99,0,0.11,N,11
22633,34,56400,MORTGAGE,2.0,DEBTCONSOLIDATION,A,7000,7.90,0,0.12,N,10
18309,28,90000,RENT,12.0,PERSONAL,D,21000,14.54,0,0.23,N,9
13623,26,95000,MORTGAGE,10.0,PERSONAL,A,6600,7.51,0,0.07,N,4
26108,28,57936,RENT,2.0,MEDICAL,D,13650,14.42,1,0.24,N,10


#### <font color = "red">Verificando valores únicos, para realizar mapeamento das variaveis

In [136]:
df_original.person_home_ownership.unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], dtype=object)

In [137]:
df_original.loan_intent.unique()

array(['PERSONAL', 'EDUCATION', 'MEDICAL', 'VENTURE', 'HOMEIMPROVEMENT',
       'DEBTCONSOLIDATION'], dtype=object)

#### <font color = "red">Realização de mapeamento no conjunto de dados

In [138]:
# Mudando nome das variáveis
df_mapeado = df_original.rename(columns=map_titulo_variaveis)
df_mapeado.head()

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [139]:
# Traduzindo valor de 2 variaveis categóricas
df_mapeado['Tipo_Moradia'] = df_mapeado['Tipo_Moradia'].map(map_tipo_moradia)
df_mapeado['Motivo_Emprestimo'] = df_mapeado['Motivo_Emprestimo'].map(map_motivo_emprestimo)


df_mapeado['Inadimplencia_Historica'] = df_mapeado['Inadimplencia_Historica'].map(map_inadimplencia_historica)

In [140]:
df_mapeado.sample(5)

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
24256,33,70116,Hipoteca,NaN,Pessoal,B,5000,11.71,0,0.07,Não,6
30729,38,62691,Próprio,17.0,Empreendimento,C,6000,13.49,0,0.10,Sim,13
27673,27,90000,Aluguel,11.0,Pessoal,B,15600,12.69,0,0.17,Não,9
32500,55,91154,Hipoteca,15.0,Pessoal,A,10000,6.62,0,0.11,Não,28
28742,33,63996,Hipoteca,15.0,Educação,A,3200,5.42,0,0.05,Não,8


-----
# Exploração Inicial

In [141]:
df_mapeado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Idade                      32581 non-null  int64  
 1   Renda_Anual                32581 non-null  int64  
 2   Tipo_Moradia               32581 non-null  object 
 3   Anos_no_Emprego            31686 non-null  float64
 4   Motivo_Emprestimo          32581 non-null  object 
 5   Grau_Emprestimo            32581 non-null  object 
 6   Valor                      32581 non-null  int64  
 7   Taxa_Juros                 29465 non-null  float64
 8   Status_Pagamento           32581 non-null  int64  
 9   Comprometimento_Renda      32581 non-null  float64
 10  Inadimplencia_Historica    32581 non-null  object 
 11  Duracao_Historico_Credito  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


#### <font color = 'red'> Alteração dos tipos de variáveis</font>

<font color = 'red'>Status_Pagamento -> int para object (Classe)

Todos os valores int serão convertidos para float</font>

In [142]:
df_mapeado['Status_Pagamento'] = df_mapeado['Status_Pagamento'].astype("object")

In [143]:
# transformando int em float
# Nome de todas as colunas numéricas
variaveis_numericas = ['Idade','Renda_Anual','Anos_no_Emprego',
                       'Valor','Taxa_Juros','Comprometimento_Renda',
                       'Duracao_Historico_Credito']

for variaveis in variaveis_numericas:
 df_mapeado[variaveis] = df_mapeado[variaveis].astype("float")

In [145]:
df_mapeado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Idade                      32581 non-null  float64
 1   Renda_Anual                32581 non-null  float64
 2   Tipo_Moradia               32581 non-null  object 
 3   Anos_no_Emprego            31686 non-null  float64
 4   Motivo_Emprestimo          32581 non-null  object 
 5   Grau_Emprestimo            32581 non-null  object 
 6   Valor                      32581 non-null  float64
 7   Taxa_Juros                 29465 non-null  float64
 8   Status_Pagamento           32581 non-null  object 
 9   Comprometimento_Renda      32581 non-null  float64
 10  Inadimplencia_Historica    32581 non-null  object 
 11  Duracao_Historico_Credito  32581 non-null  float64
dtypes: float64(7), object(5)
memory usage: 3.0+ MB


In [146]:
df_mapeado.sample(5)

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
24940,30.0,75000.0,Aluguel,6.0,Médico,C,11200.0,13.16,0,0.15,Sim,5.0
3403,22.0,59000.0,Aluguel,2.0,Educação,B,11000.0,10.74,0,0.19,Não,2.0
4758,22.0,42000.0,Próprio,0.0,Educação,C,12000.0,NaN,0,0.29,Não,4.0
1097,23.0,47500.0,Aluguel,5.0,Médico,C,17500.0,13.61,1,0.37,Não,4.0
14772,22.0,43000.0,Aluguel,0.0,Educação,A,15000.0,8.49,1,0.35,Não,2.0


<font color = 'red'>Agora o conjunto de dados está pronto para a etapa de Análise Exploratória, Engenharia de Atributos e geração de Insights, que farei no notebook 02.

In [150]:
# Salvando DF 
df_mapeado.to_csv('../data/dados_mapeados2', sep =',', index = False, encoding = 'utf-8')